In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext rpy2.ipython

In [5]:
import numpy as np
import pandas as pd

from kaggle.house_prices import helpers
from kaggle.house_prices import missing
from kaggle.house_prices import outliers

combined_dataset = helpers.load_data()
combined_dataset = missing.fix_all(combined_dataset)
combined_dataset = outliers.remove_outliers(combined_dataset)
combined_dataset['price_log'] = np.log(combined_dataset['SalePrice'])
combined_dataset = combined_dataset.drop(['SalePrice', 'Id'], axis=1)
combined_dataset.shape

(2917, 81)

In [62]:
def calc_global_quantiles(df):
    return (
        df
        ['price_log']
        .dropna()
        .quantile([.25, .5, .75])
    )
    
calc_global_quantiles(combined_dataset)

0.25    11.774713
0.50    12.001505
0.75    12.273731
Name: price_log, dtype: float64

In [22]:
def calc_ratings(df, categ_vars, target_var):
    """ XXX """
    df = (
        df[categ_vars + [target_var]]
        .dropna(subset=[target_var])
    )
    return pd.melt(
        frame=df,
        id_vars=[target_var],
        var_name='var', 
        value_name='value'
    )
    
x = (
    calc_ratings(combined_dataset, ['Alley', 'BldgType'], 'price_log')
    .groupby(['var', 'value'])
    .head()
    .sort_values(['var', 'value'])
)

In [47]:
y = x.groupby(['var', 'value'])
y.get_group(('Alley', 'Grvl'))

,price_log,var,value
21,11.845103,Alley,Grvl
79,11.608236,Alley,Grvl
92,12.004568,Alley,Grvl
106,11.512925,Alley,Grvl
204,11.608236,Alley,Grvl


In [58]:
from statsmodels.distributions.empirical_distribution import ECDF

ecdf = ECDF(y.get_group(('Alley', 'Grvl'))['price_log'])
ecdf(12.1)

1.0